# Deploy with `kubectl`

```{note}
`kubectl` is required to deploy the application. Documentation on installing `kubectl` for your OS can be found on the kubernetes website [here](https://kubernetes.io/docs/tasks/tools/).
```

`kubectl` can deploy k8s objects that are defined in .yaml (.yml) file(s). `kubectl` converts the YAML syntax into JSON when making requests to the k8s API so both formats can be used. For the examples provided in this documentation we will focus on YAML syntax as it's more human readable than JSON. The [Objects in Kubernetes](https://kubernetes.io/docs/concepts/overview/working-with-objects/) documentation goes in to much greater detail. This document will focus only on what is required to host a web application on the NCAR | CISL K8s cluster.

## Create a k8s .yml manifest

3 different kinds of k8s objects need to be used in order to deploy a web application that is accessible publicly on your browser. A [Deployment](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/), a [Service](https://kubernetes.io/docs/concepts/services-networking/service/), and an [Ingress](https://kubernetes.io/docs/concepts/services-networking/ingress/). The previous links will cover each one of these objects in much greater detail.

```{note}
All of these objects can be defined in a single YAML file. This document breaks them up to explain each individually to discuss each in more detail. Use `---` on a line by itself to divide different k8s objects defined in a single YAML file. 
```

### Deployment

A Deployment is where the state of [Pods](https://kubernetes.io/docs/concepts/workloads/pods/) are provided and maintained to the specifications declared in the object definition. The following example will touch on the fields needed in the `Deployment` section of the `.yml` file

<img src="https://ncar.github.io/cisl-cloud/_static/k8s-deployment.png" width="133%">

````{margin}
```{dropdown} apiVersion
Define the [Kubernetes API](https://kubernetes.io/docs/reference/kubernetes-api/) version to use for the object `kind:` declared. 
```
```{dropdown} kind:
Define the type of kubernetes object that is going to be declared.
```
```{dropdown} metadata:
Define the object metadata such as the Deployment name, labels to apply, and the k8s [namespace](https://kubernetes.io/docs/concepts/overview/working-with-objects/namespaces/) to deploy to.
```
```{dropdown} spec:
Define the resources for the k8s object.
```
```{dropdown} replicas:
Define the number of containers to run.
```
```{dropdown} selector:
Identify the set of objects to use in the deployment based on [labels](https://kubernetes.io/docs/concepts/overview/working-with-objects/labels/) set in metadata.
```
```{dropdown} template:
Define the Pod specification. In a Deployment this is a nested version of the [Pod template](https://kubernetes.io/docs/concepts/workloads/pods/#pod-templates).
```
```{dropdown} labels:
Define the labels to apply to the pods for use with the label selector.
```
```{dropdown} containers:
Define what container resources to use in the Pod.
```
```{dropdown} image:
Define the container image to deploy. By default this looks to [Docker Hub](https://hub.docker.com/)
```
```{dropdown} resources:
Define the container resource requests and limits.
```
```{dropdown} ports:
Define the ports that are exposed in the container and what ports to expose to k8s. 
```
````

### Service

A Service in k8s is a method to expose a network application that is running in one or more Pods.

There's a few important parts of the Deployment manifest to call out in more detail. The first is the `namespace:` field. This is an isolated space in k8s to group applications. You should have a dedicated namespace to deploy your applications to. If you do not have a dedicated namespace see the creating a namespace documentation [here](../Rancher/create-namespace.ipynb). The second is the `name:` field. This is where you define a unique and descriptive name for your Deployment. The third is the `image:` field and it's where you select the image to use in your deployment. The image will be pulled from Docker Hub by default but a different repository can be provided. The final field is the `containerPort:` field. This is very important to make sure the correct port to access the application is exposed to k8s for access. You typically can set this to a specific port in your applications settings, or use the default port opened by the application that is running your web service. If you are unsure what this is, typically you will see it in the output when you run your service locally i.e. `http://localhost:5001`. In this case 5001 is the `containerPort:` to expose. 

<img src="https://ncar.github.io/cisl-cloud/_static/k8s-service.png" width="133%">

### Ingress

A k8s Ingress object is one that maps an application exposed by a Service and routes access to external resources outside the k8s cluster, typically this uses HTTP(s) 

<img src="https://ncar.github.io/cisl-cloud/_static/k8s-ingress.png" width="133%">

````{margin}
```{dropdown} annotations:
[Annotations](https://kubernetes.io/docs/concepts/overview/working-with-objects/annotations/) are used to attach non-identifying metadata to the object. This means that annotations are not used to identify and select objects. In this example annotations are used to declare a configuration option to apply to the object.
```
```{dropdown} cert-manager.io:
cert-manager.io/cluster-issuer is used to provide the Ingress object information on how to request and obtain certificates as configured in the cluster. The cluster we use has a certificate issuer named `incommon` that is used to provide certificates for web applications in the `*.k8s.ucar.edu` domain.
```
```{dropdown} ingressClassName:
This is where the name of the ingress controller is provided. The cluster we use has a ingress controller named `nginx` and that is used to control the different Ingress objects deployed. 
```
```{dropdown} tls:
The `tls:` field is where we define the hostname, in `hosts:`, to use in the TLS certificate provided as well as set a `secretName` for the certificate issued. 
```
```{note}
The secretName field needs to be unique for the fully qualified domain name (FQDN) used since it is tied to the TLS certificate that is tied to the same FQDN.
```
```{dropdown} host:
This is the FQDN for the application being deployed. This name needs to be unique and also has to be in the `.k8s.ucar.edu` subdomain. The cluster utilizes [ExternalDNS](https://github.com/bitnami/charts/tree/main/bitnami/external-dns) to provision new DNS records but it is limited to the `.k8s.ucar.edu` domain.
```
```{dropdown} http:
Define how to expose the service to HTTP(S) traffic.
```
```{dropdown} paths:
[Paths](https://kubernetes.io/docs/concepts/services-networking/ingress/#path-types) are where endpoints are defined in order to access the applications. A hostname can have multiple paths defined by multiple ingresses running multiple applications. 
```
```{dropdown} backend:
Define the Service to connect the Ingress to by providing the Service name and the port exposed by the Service. This should match exactly what was included in the Service manifest. 
```
````

There's important fields to call out in the Ingress manifest as well. To start the `cert-manager.io/cluster-issuer:` is very important to make sure your application gets a valid certificate. This carries down to the `tls:` section and the `secretName:` field. The `secretName:` should be unique for the `hosts:` FQDN specified. If you are using the same FQDN with different prefixes the `secretName:` should be the one used for the initial certificate issue for the FQDN requested. Speaking of `hosts:` this is defined in two places, under `tls:` and `rules:`. The hostname used should be unique and has to be under the `*.k8s.ucar.edu` domain. The k8s cluster utilizes ExternalDNS to create new DNS records for the hostname provided and it is configured to only deploy under `*.k8s.ucar.edu`. The certificates that are issued are also only valid for FQDN's in `*.k8s.ucar.edu`. The final piece that is important to call out is the `port:` field. This should be the same in the Deployment, Service, and Ingress configuration.  